In [10]:
# Import libraries for data manipulation and ETL pipeline
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from pandarallel import pandarallel
pandarallel.initialize(verbose=0)
from tqdm import tqdm

In [11]:
# Establish API connection
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

cid = '3fda75b7146a4769b207ee44017b3abe'
secret = '2a755cb04a18406b9394dbef2f8069dd'

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager, requests_timeout=15)

In [12]:
# Connect to SQL Server
import sqlalchemy as sql
from sqlalchemy import text
import pyodbc
server = 'LAPTOP-V3754MEK' 
database = 'Spotify'

engine = sql.create_engine('mssql+pyodbc://' + server + '/' + database + '?trusted_connection=yes&driver=ODBC+Driver+13+for+SQL+Server')
conn = engine.connect()
results = engine.execute("SELECT track_uri2 FROM dim_tracks ORDER BY track_uri2")
results = pd.Series(results)
results.head()

0    (spotify:track:0002yNGLtYSYtc0X6ZnFvp)
1    (spotify:track:00039MgrmLoIzSpuYKurn9)
2    (spotify:track:0003Z98F6hUq7XxqSRM87H)
3    (spotify:track:0004ExljAge0P5XWn1LXmW)
4    (spotify:track:0005rgjsSeVLp1cze57jIN)
dtype: object

In [13]:
# Extract tracks from API
output_df = pd.DataFrame()
for row in tqdm(results[980001:1000001]):
      output = pd.DataFrame(sp.audio_features(row))
      output_df = pd.concat([output_df, output])
output_df.head()

100%|██████████| 20000/20000 [31:57<00:00, 10.43it/s]  


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.536,0.907,8,-5.324,1,0.1250,0.4190,0.000000,0.7650,0.970,169.633,audio_features,3ggrIgQhUoFFv6vKBPg4G6,spotify:track:3ggrIgQhUoFFv6vKBPg4G6,https://api.spotify.com/v1/tracks/3ggrIgQhUoFF...,https://api.spotify.com/v1/audio-analysis/3ggr...,125693,4
0,0.699,0.901,3,-4.810,0,0.0481,0.0141,0.043700,0.0411,0.446,127.999,audio_features,3GgrKLce2KSLSktzFKEzzG,spotify:track:3GgrKLce2KSLSktzFKEzzG,https://api.spotify.com/v1/tracks/3GgrKLce2KSL...,https://api.spotify.com/v1/audio-analysis/3Ggr...,229663,4
0,0.685,0.654,1,-8.181,0,0.2070,0.1070,0.000000,0.1270,0.297,75.276,audio_features,3GGRSJW9GVO9s3rskYm0O2,spotify:track:3GGRSJW9GVO9s3rskYm0O2,https://api.spotify.com/v1/tracks/3GGRSJW9GVO9...,https://api.spotify.com/v1/audio-analysis/3GGR...,153626,4
0,0.756,0.304,8,-9.562,1,0.2170,0.0191,0.036300,0.1110,0.492,85.359,audio_features,3GgrY8G2WmsXhP8zjq0yvZ,spotify:track:3GgrY8G2WmsXhP8zjq0yvZ,https://api.spotify.com/v1/tracks/3GgrY8G2WmsX...,https://api.spotify.com/v1/audio-analysis/3Ggr...,283973,4
0,0.777,0.819,8,-5.770,0,0.0294,0.1250,0.000006,0.1110,0.542,129.971,audio_features,3GGRZ7M2UBQEIVdXGaQ0gG,spotify:track:3GGRZ7M2UBQEIVdXGaQ0gG,https://api.spotify.com/v1/tracks/3GGRZ7M2UBQE...,https://api.spotify.com/v1/audio-analysis/3GGR...,222253,4


In [14]:
# Load data into SQL Server
output_df = output_df[['uri', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature']]
output_df = output_df.rename(columns={'key':'music_key', 'mode':'modal'})
output_df.to_sql("audio_features2", engine, if_exists='append', index=False)
print(f'{len(output_df)} rows imported to database.')
new_query = engine.execute('SELECT COUNT(DISTINCT uri) FROM audio_features2')
new_query = pd.Series(new_query)
print(f'App contains {new_query[0]} songs.')

20000 rows imported to database.
App contains (989959,) songs.
